# Event-log creator

In [1]:
# IMPORTS
import pandas as pd
import random
from datetime import datetime

In [2]:
# Set filepath for saving
file_path_general = 'your_filepath'

# Set date format
date_format = "%d.%m.%Y %H:%M"
new_date_format = "%Y-%m-%d %H:%M:%S"


In [3]:
# Read data
file_path = 'your_filepath/syntetic_merged_data.csv'
df = pd.read_csv(file_path ,sep=',')
df.head()

,Auftragsnummer,Versand_ID,Verladestart,Verladeende,Versand_SOLL,Lagereinheit_ID,Packplatz,Packzeitpunkt,Menge,Lagerort_von,Lagerort_nach,Startzeitpunkt,Endzeitpunkt
0,1008,80939,27.05.2022 21:25,28.05.2022 01:23,27.05.2022 21:55,231,Packplatz 03,27.05.2022 19:46,447,Wareneingang,Hochregallager,2022-05-27 05:18:00,2022-05-27 07:42:00
1,1008,80939,27.05.2022 21:25,28.05.2022 01:23,27.05.2022 21:55,231,Packplatz 03,27.05.2022 19:46,447,Hochregallager,Packplatz,2022-05-27 17:38:00,2022-05-27 18:27:00
2,1011,53941,27.05.2022 22:05,27.05.2022 23:38,27.05.2022 22:35,763,Packplatz 09,27.05.2022 21:17,165,Hochregallager,Packplatz,2022-05-27 16:25:00,2022-05-27 17:44:00
3,1011,53941,27.05.2022 22:05,27.05.2022 23:38,27.05.2022 22:35,763,Packplatz 09,27.05.2022 21:17,165,Wareneingang,Hochregallager,2022-05-27 17:16:00,2022-05-27 18:42:00
4,1018,85216,28.05.2022 01:02,28.05.2022 03:55,28.05.2022 01:32,770,Packplatz 04,28.05.2022 00:09,317,Wareneingang,Hochregallager,2022-05-27 09:37:00,2022-05-27 11:19:00


In [4]:
# Set process activities
process_activities = ['Ware vereinnahmen', 'Waren lagern', 'Waren kommissionieren', 'Waren im Warenausgang', 'Waren verladen']

# Set storage location
storage_locations = ['Wareneingang', 'Hochregallager', 'Packplatz', 'Warenausgang', 'Transportmittel']

In [5]:
# Make a list with all Auftragsnummern 
auftragsnummern = df['Auftragsnummer'].unique().tolist()

# Group data after Auftragsnummer
grouped_data = df.groupby('Auftragsnummer') 

# Make a new pandas dataframe
df_new = pd.DataFrame(columns=['case_id', 'process_activity', 'Auftragsnummer', 'Versand_ID', 'Verladestart', 'Verladeende', 'Versand_SOLL', 'Lagereinheit_ID', 'Packplatz', 'Packzeitpunkt', 'Menge', 'Lagerort_von', 'Lagerort_nach', 'Startzeitpunkt', 'Endzeitpunkt', 'Verspaetung', 'Lagerort'])

In [6]:
%%capture
# Restrictions:
#   - n * 'Auftragsnummern' can shiped in n * parcels ('Versand_ID'). 
#   - But only 1x parcel ('Versand_ID') can only include 1x 'Auftragsnummer'

# Get the WMS-movement rows for a unique event
# A unique event is defined by a 'Versand_ID' and 'Auftragsnummer'. Because of the restriction that 1x parcel can only include 1x 'Auftragsnummer',
# the relationship is 1 to 1 between #Versand_ID' and 'Auftragsnummer'. This is also the reason, why the 'Versand_ID' builds the Case_ID.
# The case_id shows all related WMS-movements.

# For the above descripted reason, there are two nested for-loops. Every loop groupes the dataframe further.
# The first input dataframe is a df grouped by 'Auftragsnummer', the second is grpuped by 'Versand_ID'.
# Lastly we get every time two WMS-movements and creates due to this the needed rows for the event-log. 

for i in auftragsnummern:
    case_data1 = grouped_data.get_group(i)
    versand_ids = case_data1['Versand_ID'].unique().tolist() #unique() kann entfernt werden, da Versand IDs sowieso einzigartig sind
    grouped_versand_ids = case_data1.groupby('Versand_ID')
        
    for b in versand_ids:
        how = grouped_versand_ids.get_group(b)
        case_data = grouped_versand_ids.get_group(b).sort_values(by='Lagerort_von', ascending=False).reset_index(drop=True)
        print(case_data)

        num_new_dfs = len(case_data) // 2
        
        # Create new dataframes with the length of only two elements. Every 'Versand_ID' has only one 'Auftragsnummer', because of the restriction described above. 
        # But one Auftrag can shiped in different parcels. Furthermore different articels ('Lagereinheiten_ID') can picked in one parcel ('Versand_ID').
        # Thats the reason why we got not every time only two elements after we grouped by 'Versand_ID'. So we have to group further to seperate the not belonging WMS-movements.
        # That is done by take the related WMS-movements with a for and while-loop from the pre-grouped and sorted case_data.
        for i in range(num_new_dfs):
          new_case_data_df = pd.DataFrame()
          choose_elements = i
          c = 0
          while c < 2:
            new_case_data_df = new_case_data_df.append(case_data.iloc[choose_elements], ignore_index=True)
            choose_elements = choose_elements + num_new_dfs
            c = c+1
          
          print(new_case_data_df)

        # Create case_id from 'Versand_ID'
        case_id = str(b) 

        # Get current Index
        current_index = new_case_data_df.index[0]

        # Check if the process have a delay
        if new_case_data_df.at[current_index, 'Versand_SOLL'] < new_case_data_df.at[current_index, 'Verladeende']:
          delay = True
        else:
          delay = False

        # Choose current row
        current_row = new_case_data_df.iloc[0]  

        current_row_copy = current_row.copy()

        # Get values from current row
        cu_lagerort_nach = current_row['Lagerort_nach']
        cu_endzeitpunkt = current_row['Endzeitpunkt']
        cu_startzeitpunkt = current_row['Startzeitpunkt']

        # Get values from the directly following row
        next_row = new_case_data_df.iloc[1]
        ne_startzeitpunkt = next_row['Startzeitpunkt']
        ne_endzeitpunkt = next_row['Endzeitpunkt']

        # Prepare new line to implement
        current_row_copy['Lagerort_von'] = cu_lagerort_nach
        current_row_copy['Startzeitpunkt'] = cu_endzeitpunkt
        current_row_copy['Endzeitpunkt'] = ne_startzeitpunkt
        current_row_copy['case_id'] = case_id
        current_row_copy['process_activity'] = process_activities[1]
        current_row_copy['Verspaetung'] = delay
        current_row_copy['Lagerort'] = storage_locations[1]

        # Prepare lines
        # Modify current line
        new_case_data_df.at[current_index, 'case_id'] = case_id
        new_case_data_df.at[current_index, 'process_activity'] = process_activities[0]
        new_case_data_df.at[current_index, 'Verspaetung'] = delay
        new_case_data_df.at[current_index, 'Lagerort'] = storage_locations[0]

        # Modify the next following line
        new_case_data_df.at[current_index+1, 'case_id'] = case_id
        new_case_data_df.at[current_index+1, 'process_activity'] = process_activities[2]
        new_case_data_df.at[current_index+1, 'Verspaetung'] = delay
        new_case_data_df.at[current_index+1, 'Lagerort'] = storage_locations[2]

        # Paste the copied line below the current line
        df2 = pd.concat([new_case_data_df.iloc[:current_index+1], current_row_copy.to_frame().T, new_case_data_df.iloc[current_index+1:]]).reset_index(drop=True)

        # Append activities 'Waren im Warenausgang' 
        # Get needed values from current row
        cu_wareneingang_start = new_case_data_df.loc[current_index+1, 'Endzeitpunkt']

        parsed_date_end = datetime.strptime(current_row['Verladestart'], date_format)
        formatted_date_end = parsed_date_end.strftime(new_date_format)
        cu_wareneingang_ende = formatted_date_end
        

        current_row_copy['case_id'] = case_id
        current_row_copy['Startzeitpunkt'] = cu_wareneingang_start
        current_row_copy['Endzeitpunkt'] = cu_wareneingang_ende
        current_row_copy['process_activity'] = process_activities[3]
        current_row_copy['Verspaetung'] = delay
        current_row_copy['Lagerort'] = storage_locations[3]
        current_row_copy['Lagerort_von'] = 'Packplatz'
        current_row_copy['Lagerort_nach'] = 'Warenausgang'

        df2 = df2.append(current_row_copy, ignore_index=True)

        # Append activities 'Waren verladen'
        cu_verladestart = formatted_date_end
        parsed_date_end2 = datetime.strptime(current_row['Verladeende'], date_format)
        formatted_date_end2 = parsed_date_end2.strftime(new_date_format)
        cu_verladeende = formatted_date_end2

        # Current_row_copy2['case_id'] = case_id
        current_row_copy['Startzeitpunkt'] = cu_verladestart
        current_row_copy['Endzeitpunkt'] = cu_verladeende
        current_row_copy['process_activity'] = process_activities[4]
        current_row_copy['Verspaetung'] = delay
        current_row_copy['Lagerort'] = storage_locations[4]
        current_row_copy['Lagerort_von'] = 'Warenausgang'
        current_row_copy['Lagerort_nach'] = 'Transportmittel'

        df2 = df2.append(current_row_copy, ignore_index=True)


        df_new = df_new.append(df2, ignore_index=True)

        
  
    
file_path = file_path_general + 'event-log.csv'
df_new.to_csv(file_path, index=False)
    

In [7]:
# Short event-log
# Creates rows only with the needed informations
df_short = df_new.copy()
df_short.drop(['Versand_ID', 'Verladestart', 'Verladeende', 'Versand_SOLL', 'Lagereinheit_ID', 'Packplatz', 'Packzeitpunkt'], axis=1, inplace=True)

file_path = file_path_general + "event-log_short.csv"
df_short.to_csv(file_path, index=False)